In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import pathlib
import random
import IPython.display as display
from tqdm import tqdm
from collections import Counter

Image Dimensions: 

Counter({((1400, 1050), 'drone'): 835,
         ((1400, 1050), 'Bird+2_Blade_rotor'): 815,
         ((1400, 1050), '2_blade_rotor'): 800,
         ((1400, 1050), '3_short_blade_rotor'): 800,
         ((700, 525), 'Bird'): 800,
         ((1400, 1050), '3_long_blades_rotor'): 799})
         
trim(PIL): 

Counter({(1087, 855): 1649,
         (1086, 855): 1045,
         (1087, 856): 895,
         (542, 428): 800,
         (1086, 856): 315,
         (1087, 857): 106,
         (1087, 858): 35,
         (1086, 857): 3,
         (1086, 858): 1})


In [ ]:
from PIL import Image, ImageChops

def trim(im):
    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    if bbox:
        return im.crop(bbox)

data_root = pathlib.Path('/Users/jackvittori/Desktop/uav-classification/images')

tot = []
for i, path in tqdm(enumerate([str(path) for path in data_root.glob('*/*')])):
    
    im = Image.open(path) 
    im2 = trim(im)
    tot.append(im2.size)
    
Counter(tot)

In [34]:
import tensorflow as tf
from typing import Tuple
import matplotlib.pyplot as plt

def trim_tensorflow(image: tf.Tensor) -> tf.Tensor:
    """Removes uniform borders from an image using TensorFlow.

    Args:
        image (tf.Tensor): Input image tensor of shape (H, W, 3).

    Returns:
        tf.Tensor: Cropped image tensor.
    """
    # Convert to grayscale to detect edges
    grayscale = tf.image.rgb_to_grayscale(image)

    # Create a mask of non-zero pixels (assumes background is uniform)
    mask = tf.math.reduce_any(grayscale < 0.99, axis=-1)  # Assuming background is near white

    # Get bounding box coordinates
    coords = tf.where(mask)
    ymin, xmin = tf.reduce_min(coords, axis=0)[:2]
    ymax, xmax = tf.reduce_max(coords, axis=0)[:2]

    # Crop image based on detected bounding box
    trimmed_image = tf.image.crop_to_bounding_box(image, ymin, xmin, ymax - ymin, xmax - xmin)
    
    return trimmed_image

def load_and_preprocess_image(image_path: str, resize: Tuple[int, int] = (1050, 1400)) -> tf.Tensor:
    """Loads, trims, resizes, and normalizes an image using TensorFlow.

    Args:
        image_path (str): Path to the image file.
        resize (Tuple[int, int], optional): Target size (height, width). Defaults to (1050, 1400).

    Returns:
        tf.Tensor: Preprocessed image tensor with shape (height, width, 3) and values normalized to [0, 1].
    """
    # Load and decode image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)  # Normalize to [0, 1]

    # Trim borders
    image = trim_tensorflow(image)

    # Resize to target dimensions
    image = tf.image.resize(image, resize)

    return image

for i, image_path in tqdm(enumerate([str(path) for path in data_root.glob('*/*')])):
    processed_image = load_and_preprocess_image(image_path)

    # Convert the tensor to a NumPy array
    image_np = processed_image.numpy()
    
    # Display the image
    plt.imshow(image_np)
    plt.axis("off")  # Hide axis
    plt.show()
    break

0it [00:00, ?it/s]


InvalidArgumentError: cannot compute Pack as input #1(zero-based) was expected to be a int32 tensor but is a int64 tensor [Op:Pack] name: stack

In [ ]:
data_root = pathlib.Path('/Users/jackvittori/Desktop/uav-classification/images')

tot = []
for i, path in tqdm(enumerate([str(path) for path in data_root.glob('*/*')])):
    
    im = Image.open(path) 
    im2 = trim(im)
    type(im2)
    break

In [ ]:
im2.size